<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 4. Evaluate

In [1]:
import glob
import os
import re
import subprocess

import numpy as np
import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [2]:
data_dir = 'ctc-data'
models = [
    'Fluo-N2DL-HeLa'
]
seg_dirs = ['gt-seg', 'pred-seg']

pattern = re.compile('\d{3}_GT')

node_match_threshold = 0.6

ctc_software = '../CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In [3]:
benchmarks = []

for seg_dir, seg_type in zip(seg_dirs, ['GT', 'Pred']):
    for m in models:
        print(f'------ {m} ------')
        res_subdir = os.path.join(data_dir, seg_dir, m)
        data_ids = [f.split('_')[0] for f in os.listdir(res_subdir) if pattern.fullmatch(f)]

        for data_id in data_ids:
            results = {
                'model': f'CellTrackerGNN {m} - {seg_type}',
                'data_id': data_id
            }
            gt_dir = os.path.join(res_subdir, f'{data_id}_GT/TRA')
            res_dir = os.path.join(res_subdir, f'{data_id}_RES')

            # Deepcell division benchmarking
            try:
                metrics = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir)
                results.update(metrics.stats)
            except ValueError:
                print('Issue with deepcell benchmarking of', data_id)

           # CTC metrics
            for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
                p = subprocess.run([os.path.join(ctc_software, operating_system, path), res_subdir, data_id, num_digits],
                                   stdout=subprocess.PIPE)
                outstring = p.stdout

                try:
                    val = float(outstring.decode('utf-8').split()[-1])
                    results[metric] = val
                except:
                    print('Benchmarking failure', path, m, data_id)
                    print(outstring.decode('utf-8'))

            benchmarks.append(results)

df = pd.DataFrame(benchmarks)
df.to_csv('ctc-benchmarks.csv')

------ Fluo-N2DL-HeLa ------
missed node 1_23 division completely
8_13 out degree = 3, daughters mismatch.
11_43 out degree = 4, daughters mismatch.
12_25 out degree = 3, daughters mismatch.
17_58 out degree = 4, daughters mismatch.
19_10 out degree = 4, daughters mismatch.
22_33 out degree = 3, daughters mismatch.
missed node 24_51 division completely
missed node 32_77 division completely
37_23 out degree = 5, daughters mismatch.
38_17 out degree = 3, daughters mismatch.
47_16 out degree = 3, daughters mismatch.
48_9 out degree = 3, daughters mismatch.
missed node 50_9 division completely
53_9 out degree = 4, daughters mismatch.
55_13 out degree = 3, daughters mismatch.
missed node 57_15 division completely
missed node 58_27 division completely
missed node 59_17 division completely
62_5 out degree = 3, daughters mismatch.
82_10 out degree = 3, daughters mismatch.
69_59 out degree = 3, daughters mismatch.
missed node 79_73 division completely
missed node 80_78 division completely
misse